In [1]:
import tenseal as ts
import sys

In [2]:
import base64

def writeCkks(ckks_vec, filename):
    ser_ckks_vec = base64.b64encode(ckks_vec)

    with open(filename, 'wb') as f:
        f.write(ser_ckks_vec)

def readCkks(filename):
    with open(filename, 'rb') as f:
        ser_ckks_vec = f.read()
    
    return base64.b64decode(ser_ckks_vec)

In [8]:
# client side

def ckks_context():
    context = ts.context(ts.SCHEME_TYPE.CKKS, 8192, coeff_mod_bit_sizes=[60, 40, 40])
    context.global_scale = pow(2, 40)
    return context

context = ckks_context()
context.generate_galois_keys()

public_context = context.serialize(save_public_key=True, save_secret_key=False, save_galois_keys=False, save_relin_keys=True)
writeCkks(public_context, "public_context")

In [12]:
# client has one query in form of tensor, encrypt it, and serializes it.
a = ts.plain_tensor([[1,1,1]])
enc_a = ts.ckks_tensor(context, a)
enc_as = enc_a.serialize()

# server reads the context and the client encrypted query and deserializes.
server_context = readCkks("public_context")
server_context = ts.context_from(server_context)
enc_asd = ts.lazy_ckks_tensor_from(enc_as)
enc_asd.link_context(server_context)

# server has 1000 plaintexts, it calculates dot product as sum of product, as supported (.dot is not working)
b = [[x, y, z] for x, y, z in zip(range(1000), range(1000), range(1000))]
c = enc_asd.mul(b).sum(1)

# server serializes output, and able to serialize whole output in one file.
c_s = c.serialize()
writeCkks(c_s, "cts")

# client reads output, deserializes and reads output.
c_sr = readCkks("cts")
c_sd = ts.lazy_ckks_tensor_from(c_sr)
c_sd.link_context(context)

print(c_sd.decrypt().tolist())

[2.7837114183114177e-10, 3.0000020128427467, 6.000004023172963, 9.000006037258682, 12.000008049805983, 15.000010063572537, 18.000012073340223, 21.00001408577931, 24.00001609929797, 27.000018110402102, 30.000020121779578, 33.000022135524844, 36.000024147114374, 39.00002616032425, 42.00002817322743, 45.000030184699234, 48.000032197140584, 51.000034210007534, 54.000036223561835, 57.00003823461711, 60.000040246632395, 63.000042257840654, 66.00004427150802, 69.00004628414872, 72.00004829611743, 75.00005030798052, 78.00005231928284, 81.00005433140439, 84.00005634459998, 87.00005835693435, 90.00006036999564, 93.00006238211807, 96.00006439444098, 99.00006640647878, 102.00006841847168, 105.00007043095196, 108.00007244219152, 111.00007445503569, 114.00007646737244, 117.00007848068185, 120.000080491468, 123.00008250579779, 126.00008451709775, 129.0000865289071, 132.00008854175803, 135.00009055475215, 138.00009256558627, 141.00009457938611, 144.0000965912521, 147.0000986042918, 150.00010061604593,